# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [1]:
!pip install --quiet pandas
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

![fantasy team](./img/fantasy.png)

What represents the "items" in the knapsack? 

- The drivers and constructors (teams) we can select.

What represents the weights?

- The cost of each driver/constructor (in millions of dollars).

What represents the values?

- The points each driver/constructor earns.

In [2]:
# Load the data from the CSV files
drivers_df = pd.read_csv('drivers.csv')
constructors_df = pd.read_csv('constructors.csv')

# Display the data
print("Drivers:")
display(drivers_df)

print("\nConstructors:")
display(constructors_df)

Drivers:


,Driver,Points,Cost
0,Lando Norris,100,29.6
1,Andrea Kimi Antonelli,61,19.3
2,George Russell,60,21.4
3,Max Verstappen,59,28.6
4,Oscar Piastri,55,23.0
5,Lance Stroll,33,9.3
6,Esteban Ocon,32,8.1
7,Alexaner Albon,28,12.8
8,Nico Hulkenberg,22,7.6
9,Oliver Bearman,22,6.7



Constructors:


,Constructor,Points,Cost
0,McLaren,172,30.6
1,Mercedes,136,23.3
2,Red Bull Racing,86,25.4
3,Haas,61,8.2
4,Williams,36,13.5
5,Racing Bulls,32,8.0
6,Kick Sauber,15,6.2
7,Ferrari,13,27.1
8,Aston Martin,5,7.3
9,Alpine,-20,8.3


# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem

$\text{maximize} \quad \sum_{i=1}^{n} p_i x_i$ \
$\text{subject to:}$ \
$\quad \sum_{i=1}^{n} c_i x_i \leq 50 \quad \text{(budget constraint)}$ \
$\quad \sum_{i=1}^{n} x_i = 5 \quad \text{(exactly 5 drivers)}$ \
$\quad x_i \in {0, 1} \quad \forall i \in {1, 2, \ldots, n} \quad \text{(binary decision variables)}$


## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [3]:
budget = 50 # in millions
max_drivers = 5

def check_cost_constraint(selected_drivers, drivers_df):
    """Check if selected drivers are within budget"""
    total_cost = sum(drivers_df.loc[drivers_df['Driver'].isin(selected_drivers), 'Cost'])
    return total_cost <= budget

def check_driver_count_constraint(selected_drivers):
    """Check if we have exactly 5 drivers"""
    return len(selected_drivers) == max_drivers

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [4]:
def value_to_cost(items_df): 
    """Calculate value-to-cost ratio for items"""
    items_df = items_df.copy()
    items_df['Ratio'] = items_df['Points'] / items_df['Cost']
    return items_df

def sort_items(items_df): 
    """Sort items by value-to-cost ratio in descending order"""
    items_df = value_to_cost(items_df)
    return items_df.sort_values('Ratio', ascending=False).reset_index(drop=True)

def greedy_driver_select(drivers_df):
    """Basic greedy algorithm for driver selection"""
    sorted_drivers = sort_items(drivers_df)

    selected_drivers = []
    total_cost = 0
    total_points = 0

    for _, row in sorted_drivers.iterrows():
        if total_cost + row['Cost'] <= budget and len(selected_drivers) < max_drivers:
            selected_drivers.append(row['Driver'])
            total_cost += row['Cost']
            total_points += row['Points']

    return {
        'selected_drivers': selected_drivers,
        'total_cost': total_cost,
        'total_points': total_points
    }


Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [5]:
def redux_greedy_driver_select(drivers_df):
    """Redux greedy algorithm (2-approximation) for driver selection"""
    greedy_solution = greedy_driver_select(drivers_df)
    max_value_driver = drivers_df[drivers_df['Cost'] <= budget].sort_values('Points', ascending=False).iloc[0]

    if max_value_driver['Points'] > greedy_solution['total_points']:
        return {
            'selected_drivers': [max_value_driver['Driver']],
            'total_cost': max_value_driver['Cost'],
            'total_points': max_value_driver['Points']
        }
    else:
        return greedy_solution


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 

$\text{maximize} \quad \sum_{i=1}^{n} p_i x_i + \sum_{j=1}^{m} p_j y_j$ \
$\text{subject to:}$ \
$\sum_{i=1}^{n} c_i x_i + \sum_{j=1}^{m} c_j y_j \leq 100$ \
$\sum_{i=1}^{n} x_i = 5$ \
$\sum_{j=1}^{m} y_j = 2$ \
$x_i, y_j \in \{0, 1\} \quad \forall i,j$

## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [6]:
budget = 100 # in millions
max_drivers = 5
max_constructors = 2

def check_cost_constraint(selected_drivers, selected_constructors, drivers_df, constructors_df):
    """Check if selected drivers and constructors are within budget"""
    drivers_cost = sum(drivers_df.loc[drivers_df['Driver'].isin(selected_drivers), 'Cost'])
    constructors_cost = sum(constructors_df.loc[constructors_df['Constructor'].isin(selected_constructors), 'Cost'])
    total_cost = drivers_cost + constructors_cost
    return total_cost <= budget

def check_count_constraints(selected_drivers, selected_constructors):
    """Check if we have exactly 5 drivers and 2 constructors"""
    return len(selected_drivers) == max_drivers and len(selected_constructors) == max_constructors

def calculate_total_points(selected_drivers, selected_constructors, drivers_df, constructors_df):
    """Calculate total points for selected drivers and constructors"""
    drivers_points = sum(drivers_df.loc[drivers_df['Driver'].isin(selected_drivers), 'Points'])
    constructors_points = sum(constructors_df.loc[constructors_df['Constructor'].isin(selected_constructors), 'Points'])
    return drivers_points + constructors_points

def calculate_total_cost(selected_drivers, selected_constructors, drivers_df, constructors_df):
    """Calculate total cost for selected drivers and constructors"""
    drivers_cost = sum(drivers_df.loc[drivers_df['Driver'].isin(selected_drivers), 'Cost'])
    constructors_cost = sum(constructors_df.loc[constructors_df['Constructor'].isin(selected_constructors), 'Cost'])
    return drivers_cost + constructors_cost

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [7]:
def greedy_f1(drivers_df, constructors_df):
    """Greedy algorithm for selecting both drivers and constructors"""

    drivers_with_ratio = value_to_cost(drivers_df)
    constructors_with_ratio = value_to_cost(constructors_df)

    drivers_with_ratio['Type'] = 'Driver'
    constructors_with_ratio['Type'] = 'Constructor'

    all_items = pd.concat([drivers_with_ratio, constructors_with_ratio])
    all_items_sorted = all_items.sort_values('Ratio', ascending=False).reset_index(drop=True)

    selected_drivers = []
    selected_constructors = []

    for _, item in all_items_sorted.iterrows():
        if item['Type'] == 'Driver' and len(selected_drivers) < max_drivers:
            temp_drivers = selected_drivers + [item['Driver']]
            if check_cost_constraint(temp_drivers, selected_constructors, drivers_df, constructors_df):
                selected_drivers.append(item['Driver'])
        
        elif item['Type'] == 'Constructor' and len(selected_constructors) < max_constructors:
            temp_constructors = selected_constructors + [item['Constructor']]
            if check_cost_constraint(selected_drivers, temp_constructors, drivers_df, constructors_df):
                selected_constructors.append(item['Constructor'])

        if check_count_constraints(selected_drivers, selected_constructors):
            break

    total_cost = calculate_total_cost(selected_drivers, selected_constructors, drivers_df, constructors_df)
    total_points = calculate_total_points(selected_drivers, selected_constructors, drivers_df, constructors_df)

    return {
        'selected_drivers': selected_drivers,
        'selected_constructors': selected_constructors,
        'total_cost': total_cost,
        'total_points': total_points
    }